In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Data base import

In [3]:
from utils import sample_data

nb_samples = 10
path_save = Path("../data/samples/sample_{}.csv".format(nb_samples))
path_csv = Path("../data/train.csv")

SMALL_DATA = sample_data.SmallData(
    path_csv, nb_samples=nb_samples,
    save=True, path_save=path_save
)

In [4]:
print(SMALL_DATA.small_data["sentiment"])

20099   -1
2767     0
21868    0
18013    1
13465    1
19904   -1
24875    0
15091    0
11472    1
15483   -1
Name: sentiment, dtype: int64


# Pre-processing

In [ ]:
# TODO

# Classification

In [6]:
from sklearn.neighbors import  KNeighborsRegressor

In [ ]:
nb_neighbors = 2

knn = KNeighborsRegressor(nb_neighbors, weights="distance")
knn.fit(X_train, Y_train)

In [ ]:
from utils.post-processing import pred_to_sentance, sentance_to_string, filter_character

pred = knn.predict(X_val)
meaning_sentences = pred_to_sentance(pred)
results = [filter_character(" ".join(sentaence), "$") for sentence]

In [ ]:
from utils.loss import jaccard

avg = 0
for i in range(len(results)):
    avg += jaccard(results[i], meaning_val[i])